In [104]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import sqlite3
import pandas as pd


In [36]:
# Create engine using the sqlite database file
engine = create_engine("sqlite:///./database/project3_group6.sqlite")

In [37]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['death_rates', 'fast_food', 'us_states', 'gdp_state', 'overweight_obesity']

In [38]:
# Map classes
death_rates = Base.classes.death_rates
fast_food =  Base.classes.fast_food
us_states =  Base.classes.fast_food
gdp_state =  Base.classes.gdp_state
overweight_obesity =  Base.classes.overweight_obesity

In [39]:
# create a session
session = Session(engine)

In [40]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [41]:
inspector.get_table_names()

['death_rates',
 'fast_food',
 'gdp_state',
 'overweight_obesity',
 'sqlite_sequence',
 'us_states']

In [49]:
columns = inspector.get_columns('overweight_obesity')
for column in columns:
    print(column["name"], column["type"])

ID INTEGER
date_year INTEGER
state_code TEXT
response TEXT
break_out TEXT
break_out_category TEXT
sample_size INTEGER
data_value FLOAT
latitude FLOAT
longitude FLOAT


In [43]:
columns = inspector.get_columns('us_states')
for column in columns:
    print(column["name"], column["type"])

state_code TEXT
state_name TEXT


In [46]:
fast_food_df = pd.read_sql_table('fast_food', engine)
us_states_df = pd.read_sql_table('us_states', engine)


In [48]:
merged_df = pd.merge(fast_food_df, us_states_df, on='state_code')
merged_df.head()

,ID,state_code,counts,state_name
0,1,CA,1201,California
1,2,TX,811,Texas
2,3,FL,621,Florida
3,4,OH,522,Ohio
4,5,GA,420,Georgia


In [50]:
overweight_obesity_df = pd.read_sql_table('overweight_obesity', engine)

In [53]:
overweight_obesity_df.head(20)

,ID,date_year,state_code,response,break_out,break_out_category,sample_size,data_value,latitude,longitude
0,1,2019,DE,Obese (BMI 30.0 - 99.8),55-64,Age Group,299,40.1,39.008831,-75.577741
1,2,2019,KS,Obese (BMI 30.0 - 99.8),25-34,Age Group,386,34.3,38.347740,-98.200781
2,3,2019,LA,Obese (BMI 30.0 - 99.8),55-64,Age Group,395,41.9,31.312661,-92.445680
3,4,2019,KY,Overweight (BMI 25.0-29.9),35-44,Age Group,294,35.6,37.645970,-84.774971
4,5,2019,NY,Overweight (BMI 25.0-29.9),45-54,Age Group,713,37.2,42.827001,-75.543970
5,6,2019,PA,Obese (BMI 30.0 - 99.8),18-24,Age Group,94,23.3,40.793730,-77.860700
6,7,2019,GU,Obese (BMI 30.0 - 99.8),18-24,Age Group,39,22.2,13.444304,144.793731
7,8,2014,GU,Obese (BMI 30.0 - 99.8),25-34,Age Group,148,34.6,13.444304,144.793731
8,9,2014,IN,Overweight (BMI 25.0-29.9),25-34,Age Group,249,32.8,39.766910,-86.149960
9,10,2014,PR,Overweight (BMI 25.0-29.9),45-54,Age Group,401,42.7,18.220833,-66.590149


In [69]:
# Extract unique state codes with latitude and longitude
state_lat_long_df = overweight_obesity_df[['state_code', 'latitude', 'longitude']].drop_duplicates()

# Merge with merged_df using 'state_code'
final_merged_df = pd.merge(merged_df, state_lat_long_df, on='state_code')
final_merged_df = final_merged_df = final_merged_df.drop_duplicates(subset=['ID'])
final_merged_df = final_merged_df.reset_index(drop=True)


In [71]:
final_merged_df.head(50)

,ID,state_code,counts,state_name,latitude,longitude
0,1,CA,1201,California,37.638640,-121.000000
1,2,TX,811,Texas,31.827240,-99.426770
2,3,FL,621,Florida,28.932040,-81.928961
3,4,OH,522,Ohio,40.060210,-82.404260
4,5,GA,420,Georgia,32.839681,-83.627580
5,6,IL,405,Illinois,40.485010,-88.997710
6,7,PA,383,Pennsylvania,40.793730,-77.860700
7,8,MI,374,Michigan,44.661320,-84.714390
8,9,NY,352,New York,42.827001,-75.543970
9,10,AZ,330,Arizona,34.865970,-111.763811


In [76]:
# Convert 'date_year' to string to ensure it's treated as a categorical variable
overweight_obesity_df['date_year'] = overweight_obesity_df['date_year'].astype(str)

# Filter data for the year 2019
df_2019 = overweight_obesity_df[overweight_obesity_df['date_year'] == '2019']

# Group by 'state_code' and 'break_out', then calculate the average 'data_value'
average_data_value_2019 = df_2019.groupby(['state_code', 'break_out'])['data_value'].mean().reset_index()

# Group by 'state_code' and calculate the overall average 'data_value'
overall_average_data_value_2019 = average_data_value_2019.groupby('state_code')['data_value'].mean().reset_index()
# Display the result
print(overall_average_data_value_2019)



   state_code  data_value
0          AK   33.008333
1          AL   34.666667
2          AR   35.041667
3          AZ   32.633333
4          CA   30.958333
5          CO   29.050000
6          CT   32.333333
7          DC   28.000000
8          DE   33.883333
9          FL   31.441667
10         GA   33.091667
11         GU   32.975000
12         HI   28.866667
13         IA   33.808333
14         ID   31.691667
15         IL   32.275000
16         IN   34.250000
17         KS   34.783333
18         KY   35.591667
19         LA   35.166667
20         MA   29.400000
21         MD   32.741667
22         ME   31.858333
23         MI   34.625000
24         MN   32.208333
25         MO   33.600000
26         MS   36.150000
27         MT   31.825000
28         NC   34.450000
29         ND   35.075000
30         NE   34.125000
31         NH   33.283333
32         NM   32.908333
33         NV   33.341667
34         NY   31.066667
35         OH   34.141667
36         OK   35.441667
37         O

In [97]:
merged_df_with_avg_obesity_2019 = pd.merge(final_merged_df, overall_average_data_value_2019, on='state_code', how='left')
merged_df_with_avg_obesity_2019

,ID,state_code,counts,state_name,latitude,longitude,data_value
0,1,CA,1201,California,37.638640,-121.000000,30.958333
1,2,TX,811,Texas,31.827240,-99.426770,34.583333
2,3,FL,621,Florida,28.932040,-81.928961,31.441667
3,4,OH,522,Ohio,40.060210,-82.404260,34.141667
4,5,GA,420,Georgia,32.839681,-83.627580,33.091667
5,6,IL,405,Illinois,40.485010,-88.997710,32.275000
6,7,PA,383,Pennsylvania,40.793730,-77.860700,33.558333
7,8,MI,374,Michigan,44.661320,-84.714390,34.625000
8,9,NY,352,New York,42.827001,-75.543970,31.066667
9,10,AZ,330,Arizona,34.865970,-111.763811,32.633333


In [99]:
# NJ doesnot have data for 2019 -> drop that row
merged_df_with_avg_obesity_2019 = merged_df_with_avg_obesity_2019.dropna()
merged_df_with_avg_obesity_2019

,ID,state_code,counts,state_name,latitude,longitude,data_value
0,1,CA,1201,California,37.638640,-121.000000,30.958333
1,2,TX,811,Texas,31.827240,-99.426770,34.583333
2,3,FL,621,Florida,28.932040,-81.928961,31.441667
3,4,OH,522,Ohio,40.060210,-82.404260,34.141667
4,5,GA,420,Georgia,32.839681,-83.627580,33.091667
5,6,IL,405,Illinois,40.485010,-88.997710,32.275000
6,7,PA,383,Pennsylvania,40.793730,-77.860700,33.558333
7,8,MI,374,Michigan,44.661320,-84.714390,34.625000
8,9,NY,352,New York,42.827001,-75.543970,31.066667
9,10,AZ,330,Arizona,34.865970,-111.763811,32.633333


In [100]:
import folium
import fiona
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [101]:
#creating the base map
m= folium.Map(location = [38.27312, -98.5821872], zoom_start = 4)

#creating bubble map with restaurant counts per state.
#Loop through each row in the dataframe
for i in range(0,len(merged_df_with_avg_obesity_2019)):
   folium.Circle(
      location=[merged_df_with_avg_obesity.iloc[i]['latitude'], merged_df_with_avg_obesity_2019.iloc[i]['longitude']],
      popup=merged_df_with_avg_obesity_2019.iloc[i]['counts'],
      radius=float(merged_df_with_avg_obesity_2019.iloc[i]['counts'])*200,
      color='crimson',
      fill=True,
      fill_color='crimson'
      ).add_to(m)
    
m

In [105]:
# Add a HeatMap map to the base map for Diabeties data
lats_longs_weight = list(map(list, zip(merged_df_with_avg_obesity_2019["latitude"],
                          merged_df_with_avg_obesity_2019["longitude"],
                          merged_df_with_avg_obesity_2019["data_value"]
                         )
               )
           )

#Creating a title for the layers in map
fg = folium.FeatureGroup(name="Obesity HeatMap")
fg.add_child(HeatMap(lats_longs_weight))
m.add_child(fg)
m


In [107]:
# export results to csv
merged_df_with_avg_obesity_2019.to_csv('./clean_data/states_lat_long_avg_obesity_2019.csv') 

In [108]:
json_data = merged_df_with_avg_obesity_2019.to_json(orient="index")
json_data_records = merged_df_with_avg_obesity_2019.to_json(orient="records")

In [109]:
print(json_data_records)

[{"ID":1,"state_code":"CA","counts":1201,"state_name":"California","latitude":37.638640123,"longitude":-120.999999538,"data_value":30.9583333333},{"ID":2,"state_code":"TX","counts":811,"state_name":"Texas","latitude":31.827240407,"longitude":-99.426770206,"data_value":34.5833333333},{"ID":3,"state_code":"FL","counts":621,"state_name":"Florida","latitude":28.932040377,"longitude":-81.928960539,"data_value":31.4416666667},{"ID":4,"state_code":"OH","counts":522,"state_name":"Ohio","latitude":40.060210141,"longitude":-82.404260056,"data_value":34.1416666667},{"ID":5,"state_code":"GA","counts":420,"state_name":"Georgia","latitude":32.839681093,"longitude":-83.627580346,"data_value":33.0916666667},{"ID":6,"state_code":"IL","counts":405,"state_name":"Illinois","latitude":40.485010283,"longitude":-88.997710178,"data_value":32.275},{"ID":7,"state_code":"PA","counts":383,"state_name":"Pennsylvania","latitude":40.793730152,"longitude":-77.860700294,"data_value":33.5583333333},{"ID":8,"state_code"

In [110]:
print(json_data)

{"0":{"ID":1,"state_code":"CA","counts":1201,"state_name":"California","latitude":37.638640123,"longitude":-120.999999538,"data_value":30.9583333333},"1":{"ID":2,"state_code":"TX","counts":811,"state_name":"Texas","latitude":31.827240407,"longitude":-99.426770206,"data_value":34.5833333333},"2":{"ID":3,"state_code":"FL","counts":621,"state_name":"Florida","latitude":28.932040377,"longitude":-81.928960539,"data_value":31.4416666667},"3":{"ID":4,"state_code":"OH","counts":522,"state_name":"Ohio","latitude":40.060210141,"longitude":-82.404260056,"data_value":34.1416666667},"4":{"ID":5,"state_code":"GA","counts":420,"state_name":"Georgia","latitude":32.839681093,"longitude":-83.627580346,"data_value":33.0916666667},"5":{"ID":6,"state_code":"IL","counts":405,"state_name":"Illinois","latitude":40.485010283,"longitude":-88.997710178,"data_value":32.275},"6":{"ID":7,"state_code":"PA","counts":383,"state_name":"Pennsylvania","latitude":40.793730152,"longitude":-77.860700294,"data_value":33.5583

In [121]:
save_file_records = open("savedata_records.json", "w")  
save_file_records.write(json_data_records) 
save_file.close()  